# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [25]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Calculating the Rates of Callbacks:
---

In [16]:
# Black Rate of Callback
n1 = len(data[data.race=='b']) # number of black-sounding names
sum_b = sum(data[data.race=='b'].call) # number of callbacks for black-sounding names

rate_b = sum_b / n1

# White Rate of Callback
n2 = len(data[data.race=='w']) # number of white-sounding names
sum_w = sum(data[data.race=='w'].call) # number of callbacks for white-sounding names

rate_w = sum_w / n2

rate_b, rate_w

(0.064476386036960986, 0.096509240246406572)

1) What test is appropriate for this problem? Does CLT apply?
---

Requirements for CLT:

1) **The samples must be independent:** Black and White sounding names were randomly assigned to similar resumes so they represent a random sample adn are independent

2) **The sample size must be 'big enough'**:

The sample size must not be bigger than 10% of the entire population. With around 2400 data points each representing the black and white race in America, we can assume this would be less than 10% of the millions of each race in America. 

The sample size should be large enough so that:
    rate_b * p (# of black sounding individuals in US) >= 10
    rate_w * q (# of white sounding individuals in US) >= 10
    
*Assuming there is a mimimum of a million of each race (VERY conservative estimate):*

In [20]:
rate_b *1000000, rate_w * 1000000

(64476.386036960983, 96509.240246406567)

**CLT conditions are met.**

The appropriate test for this problem will be a **Difference in Proportion Confidence Interval and Two-Sided Z-Test**. We using the z-statistic becuase there is a high sample size. 

2. What are the null and alternate hypotheses?
---

Null Hypothesis (H0): rate_w - rate_b = 0

Alternative Hypothesis (Ha): rate_w = rate_b != 0

Significance Level = .05

3. Compute margin of error, confidence interval, and p-value.
----

In [42]:
p = rate_w - rate_b

std = math.sqrt((rate_w * (1-rate_w) / n2) + (rate_b * (1-rate_b)/n1))

z_score = 1.96
# with alpha = .05
# 95% chance that rate_w - rate_b is within 1.96 standard deviations from
# our sample proportion according to the z-table.

**Confidence Interval**

In [45]:
ME = z_score * std

lower = p - ME
upper = p + ME

print 'Margin of Error: ', ME
print 'Confidence Interval', [lower, upper]

Margin of Error:  0.0152554063499
Confidence Interval [0.016777447859559147, 0.047288260559332024]


**Hypothesis Test**

Assuming the null hypothesis is true, how many times do we see that we see rate_w - rate_b = 0? If it occurs a lot then the effect is probably due to chance. If it does not, then the effect is sigificant.

In [ ]:
#Converting proportion differences into standard deviations so we can use a score and proability: In other words what is the probability or liklihood of getting a rate_w - rate_b = 0 in standard deviations from the population difference? If it is less than 5% then significant. 

In [52]:
# New calculation for std becuase we are assuming null hypothesis is true
# Therefore must calculate population std such that p1 = p2 = p_hat
# p_hat is proportion of total callbacks for whole sample
# calculating proportion of callbacks disregarding race

p = rate_w = rate_b

p_hat = (sum_b + sum_w) / (n1 + n2)

std = math.sqrt( (2*p_hat*1-p_hat) / n1 ) #n1 = n2 so divide by either

# Calculate z-score
# How many standard devations away from the mean is our sample statistics
z_score = (p - 0) / std

z_score

11.214285714285715

P-Value with Significance value of .05

The probability of getting a z-score of 11 is very small, even while assuming the null hypothesis is true.

Therefore, with a z-score as high as 11, the **p-value = 0**

4. Write a story describing the statistical significance in the context or the original problem.
---

**The is a 95% chance that the true difference of white-sounding call back rates and black-sounding call back rates is between .016 and .04. **

This means we are 95% confident that there exists a difference between the two races in which individuals with white-sounding names are favored and called back more often.

**p-value = 0**

The probability of getting a z-score of 11 is very small, even while assuming the null hypothesis is true. This means that the effect that we see (a difference in the proportion of callbacks between white and balck sounding names) is significant. 

**Race has a significant impact on the rate of callbacks for resumes.**

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
---

No, the analysis only looked at the relationship between race adn callbacks. There are a number of other variables included in the data that may have equal or more impact on the callback rate. 

We must test teh signiicance of these variables to callback rate to determine which is the best to determine callback success.

I would amend the analysis by adding on the additional data in the table and testing for significance and coorelation.